In [1]:
import pandas as pd
import pickle
import json
import ast
import os
from tqdm import tqdm
import numpy as np
from scipy.sparse import coo_matrix
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF

In [5]:
data = pd.read_csv(r'C:\Users\frmar\OneDrive\Desktop\GitHub\Music-Recommender-System\dataframes\playlists_trackId.csv')

In [6]:
# Parse the tracks column into actual lists of integers
data["tracks"] = data["tracks"].apply(lambda x: list(map(int, ast.literal_eval(x))))

# Display the transformed DataFrame


In [7]:
print(data['tracks'])

0         [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...
1         [52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 6...
2         [91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101,...
3         [155, 156, 157, 158, 159, 160, 161, 162, 163, ...
4         [281, 282, 283, 284, 285, 286, 287, 288, 289, ...
                                ...                        
999995    [116671, 5783, 48788, 5784, 91283, 17696, 5046...
999996    [2262285, 2262286, 2262287, 2262288, 2262289, ...
999997    [39643, 41143, 5434, 22056, 3860, 1043, 9152, ...
999998    [5768, 5872, 14504, 3936, 76224, 5838, 5838, 2...
999999    [12421, 12518, 14294, 268821, 311299, 92033, 1...
Name: tracks, Length: 1000000, dtype: object


CERCO LE TRACCE PIU COMUNI

In [8]:
import pandas as pd
from collections import Counter

# Assuming 'df' is your DataFrame and 'tracks' is the column with lists of numbers
def count_occurrences(df, column_name):
    # Flatten the list of lists into a single list
    all_numbers = [num for sublist in df[column_name] for num in sublist]
    
    # Count occurrences
    number_counts = Counter(all_numbers)
    
    # Convert to DataFrame
    df_counts = pd.DataFrame(number_counts.items(), columns=['Number', 'Count'])
    
    return df_counts

# Example usage
df_counts = count_occurrences(data, "tracks")

# Display result
print(df_counts)


          Number  Count
0              1   6841
1              2  13205
2              3  16914
3              4   9066
4              5  23173
...          ...    ...
2262287  2262288      1
2262288  2262289      1
2262289  2262290      1
2262290  2262291      1
2262291  2262292      1

[2262292 rows x 2 columns]


In [9]:
# Filtering the dataframe to keep only numbers with more than 100 occurrences
df_filtered = df_counts[df_counts["Count"] > 100]

# Display the filtered dataframe
print(df_filtered)

        Number  Count
0            1   6841
1            2  13205
2            3  16914
3            4   9066
4            5  23173
...        ...    ...
585317  585318    125
591072  591073    112
601613  601614    114
631268  631269    138
790629  790630    139

[69678 rows x 2 columns]


In [10]:
# Summing the "Count" column to get the total number of occurrences
total_occurrences = df_filtered["Count"].sum()
total = 66346428
# Display the total occurrences
print("Totale canzoni considerate:", total_occurrences)
print("Percentuale dataset considerato:", total_occurrences/total*100,'%')

Totale canzoni considerate: 53466464
Percentuale dataset considerato: 80.58680114625011 %


FACCIO LA DIVISIONE TRAIN E TEST

In [11]:
# Step 3: Select Top 70K Most Popular Tracks
popular_tracks = set(df_filtered["Number"].values)

In [13]:
print(type(popular_tracks))

<class 'set'>


In [14]:
import pickle


# Save set to a file
with open("popular_tracks.pkl", "wb") as f:
    pickle.dump(popular_tracks, f)

In [2]:
# Load set from a file
with open("popular_tracks.pkl", "rb") as f:
    popular_tracks = pickle.load(f)

print(popular_tracks)

{np.int64(131072), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22), np.int64(23), np.int64(24), np.int64(25), np.int64(26), np.int64(27), np.int64(28), np.int64(29), np.int64(30), np.int64(31), np.int64(32), np.int64(33), np.int64(34), np.int64(35), np.int64(36), np.int64(37), np.int64(38), np.int64(39), np.int64(40), np.int64(41), np.int64(42), np.int64(43), np.int64(44), np.int64(45), np.int64(131109), np.int64(47), np.int64(48), np.int64(49), np.int64(50), np.int64(51), np.int64(52), np.int64(131118), np.int64(393267), np.int64(61), np.int64(62), np.int64(63), np.int64(64), np.int64(65), np.int64(66), np.int64(68), np.int64(69), np.int64(70), np.int64(71), np.int64(72), np.int64(73), np.int64(74), np.int64(75), np.int64(76), np.int64(77)

In [16]:
print(len(popular_tracks))

69678


In [29]:
# Step 4: Filter Training Data Without Exploding
def filter_popular_tracks(row):
    return [t for t in row if t in popular_tracks]
data["filtered_track_id"] = data["tracks"].apply(filter_popular_tracks)

In [31]:
data.drop(columns=["tracks"], inplace=True)

In [32]:
print(data)

        user_id                                  filtered_track_id
0             0  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...
1             1  [52, 61, 62, 63, 64, 65, 66, 68, 69, 70, 71, 7...
2             2  [94, 95, 97, 100, 101, 102, 103, 104, 105, 107...
3             3  [155, 157, 159, 160, 164, 171, 172, 173, 178, ...
4             4  [281, 282, 283, 285, 286, 287, 288, 289, 290, ...
...         ...                                                ...
999995   999995  [116671, 5783, 48788, 5784, 91283, 17696, 5046...
999996   999996  [65312, 128606, 47389, 76092, 53084, 79558, 13...
999997   999997  [39643, 41143, 5434, 22056, 3860, 1043, 9152, ...
999998   999998  [5768, 5872, 14504, 3936, 76224, 5838, 5838, 1...
999999   999999  [12421, 12518, 14294, 268821, 92033, 12463, 41...

[1000000 rows x 2 columns]


In [36]:
data.to_csv(r'C:\Users\frmar\OneDrive\Desktop\GitHub\Music-Recommender-System\dataframes\filtered_playlists_trackId2901.csv', index=False)

In [3]:
data = pd.read_csv(r'C:\Users\frmar\OneDrive\Desktop\GitHub\Music-Recommender-System\dataframes\filtered_playlists_trackId2901.csv')

# Parse the tracks column into actual lists of integers
data["filtered_track_id"] = data["filtered_track_id"].apply(lambda x: list(map(int, ast.literal_eval(x))))
data = data[data['filtered_track_id'].apply(lambda x: len(x) > 10)]

In [4]:
print(data)

        user_id                                  filtered_track_id
0             0  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...
1             1  [52, 61, 62, 63, 64, 65, 66, 68, 69, 70, 71, 7...
2             2  [94, 95, 97, 100, 101, 102, 103, 104, 105, 107...
3             3  [155, 157, 159, 160, 164, 171, 172, 173, 178, ...
4             4  [281, 282, 283, 285, 286, 287, 288, 289, 290, ...
...         ...                                                ...
999994   999994  [64015, 32808, 43052, 1043, 2126, 2131, 30857,...
999995   999995  [116671, 5783, 48788, 5784, 91283, 17696, 5046...
999996   999996  [65312, 128606, 47389, 76092, 53084, 79558, 13...
999997   999997  [39643, 41143, 5434, 22056, 3860, 1043, 9152, ...
999999   999999  [12421, 12518, 14294, 268821, 92033, 12463, 41...

[882135 rows x 2 columns]


In [4]:
import pandas as pd
import random
from tqdm import tqdm

# Funzione per suddividere ogni playlist in train e test
def split_train_test(df, train_ratio=0.8):
    train_data = []
    test_data = []

    for _, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing rows"):
        tracks = row['filtered_track_id']
        random.shuffle(tracks)  # Mescolare le canzoni in modo casuale

        # Calcolare la soglia per la divisione
        train_size = int(len(tracks) * train_ratio)

        # Creare i set di train e test
        train_tracks = tracks[:train_size]
        test_tracks = tracks[train_size:]

        # Aggiungere i dati nei rispettivi DataFrame
        for track in train_tracks:
            train_data.append((_, track))  # Associamo l'indice della playlist e il track

        for track in test_tracks:
            test_data.append((_, track))  # Associamo l'indice della playlist e il track

    # Creazione dei DataFrame
    train_df = pd.DataFrame(train_data, columns=['playlist', 'track'])
    test_df = pd.DataFrame(test_data, columns=['playlist', 'track'])



    return train_df, test_df


# Suddivisione in train e test
train_df, test_df = split_train_test(data, train_ratio = 0.95)
train_df = train_df.groupby('playlist')['track'].apply(list).reset_index()
test_df = test_df.groupby('playlist')['track'].apply(list).reset_index()

# Visualizzare i risultati
print("Train Set:")
print(train_df)

print("\nTest Set:")
print(test_df)

Processing rows:   0%|          | 0/882135 [00:00<?, ?it/s]

Processing rows: 100%|██████████| 882135/882135 [01:04<00:00, 13722.22it/s]


Train Set:
        playlist                                              track
0              0  [45, 2, 9, 12, 14, 31, 44, 43, 13, 51, 27, 25,...
1              1  [66, 84, 74, 85, 75, 64, 72, 86, 78, 65, 70, 6...
2              2  [97, 138, 110, 95, 123, 101, 132, 94, 139, 127...
3              3  [178, 182, 267, 187, 226, 159, 264, 185, 172, ...
4              4  [286, 287, 289, 291, 283, 285, 290, 297, 292, ...
...          ...                                                ...
882130    999994  [16913, 3546, 2009, 115711, 79424, 1043, 16957...
882131    999995  [5789, 32874, 17696, 29029, 5783, 76326, 78205...
882132    999996  [76342, 31026, 13866, 274083, 76331, 128606, 1...
882133    999997  [50604, 64014, 12905, 77270, 41143, 39643, 354...
882134    999999  [12535, 12512, 53263, 83428, 39988, 40000, 411...

[882135 rows x 2 columns]

Test Set:
        playlist                           track
0              0                     [50, 39, 3]
1              1                     

In [5]:
# Step 5: Create User-Item Matrix (Sparse)
user_map = {u: i for i, u in enumerate(train_df["playlist"].unique())}
track_map = {t: i for i, t in enumerate(popular_tracks)}

In [6]:
row_indices = []
col_indices = []
data_values = []

seen_pairs = set()  # To ensure binary interactions
for _, row in tqdm(train_df.iterrows()):
    user_idx = user_map[row["playlist"]]
    track_indices = [track_map[t] for t in row["track"] if t in track_map]
    for track_idx in track_indices:
        if (user_idx, track_idx) not in seen_pairs:
            row_indices.append(user_idx)
            col_indices.append(track_idx)
            data_values.append(1)  # Binary interaction
            seen_pairs.add((user_idx, track_idx))

882135it [01:49, 8025.66it/s] 


In [7]:
print(track_indices)

[10946, 10923, 39351, 54237, 31321, 31333, 32017, 10839, 68265, 10904, 31323, 49181, 49163, 10841, 10900, 3543, 57735, 10900, 42108, 10876, 31328, 49151, 46133, 10839, 10836, 60015, 12432, 10837, 18865, 10859, 10922, 68968, 10963, 12409, 18859, 49174, 32029, 31344, 10967, 31334, 10838, 49152, 10841, 10875, 10864, 18857, 66504, 54802, 12440, 10868, 10835, 10874, 10840, 48517, 35078, 35084, 11867, 32310, 54236, 49151, 42111, 31315, 10875, 10929, 49179, 32469]


In [8]:
# Ensure the matrix dimensions match the compact index sizes
user_item_matrix = csr_matrix((data_values, (row_indices, col_indices)), shape=(len(user_map), len(track_map)))
user_item_matrix = user_item_matrix.astype(float) 

In [9]:
print(user_item_matrix)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 49090181 stored elements and shape (882135, 69678)>
  Coords	Values
  (0, 1)	1.0
  (0, 2)	1.0
  (0, 4)	1.0
  (0, 5)	1.0
  (0, 6)	1.0
  (0, 7)	1.0
  (0, 8)	1.0
  (0, 9)	1.0
  (0, 10)	1.0
  (0, 11)	1.0
  (0, 12)	1.0
  (0, 13)	1.0
  (0, 14)	1.0
  (0, 15)	1.0
  (0, 16)	1.0
  (0, 17)	1.0
  (0, 18)	1.0
  (0, 19)	1.0
  (0, 20)	1.0
  (0, 21)	1.0
  (0, 22)	1.0
  (0, 23)	1.0
  (0, 24)	1.0
  (0, 25)	1.0
  (0, 26)	1.0
  :	:
  (882134, 32017)	1.0
  (882134, 32029)	1.0
  (882134, 32310)	1.0
  (882134, 32469)	1.0
  (882134, 35078)	1.0
  (882134, 35084)	1.0
  (882134, 39351)	1.0
  (882134, 42108)	1.0
  (882134, 42111)	1.0
  (882134, 46133)	1.0
  (882134, 48517)	1.0
  (882134, 49151)	1.0
  (882134, 49152)	1.0
  (882134, 49163)	1.0
  (882134, 49174)	1.0
  (882134, 49179)	1.0
  (882134, 49181)	1.0
  (882134, 54236)	1.0
  (882134, 54237)	1.0
  (882134, 54802)	1.0
  (882134, 57735)	1.0
  (882134, 60015)	1.0
  (882134, 66504)	1.0
  (882134, 68265

In [10]:
# Verifica se ci sono valori diversi da 1
unique_values = np.unique(user_item_matrix.data)
print(f"Valori unici nella matrice sparsa: {unique_values}")

Valori unici nella matrice sparsa: [1.]


In [11]:
from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix
import numpy as np

# Calcola SVD approssimato (k componenti singolari principali)
k = 50 # Numero di componenti principali desiderate
U, Sigma, VT = svds(user_item_matrix, k=k)

# Sigma sarà un array dei valori singolari principali
print("U shape:", U.shape)
print("Sigma shape:", Sigma.shape)
print("VT shape:", VT.shape)

U shape: (882135, 50)
Sigma shape: (50,)
VT shape: (50, 69678)


In [12]:
# Ordina i valori singolari in ordine decrescente
sorted_indices = np.argsort(Sigma)[::-1]
Sigma = Sigma[sorted_indices]  # Ordina Sigma
U = U[:, sorted_indices]       # Ordina le colonne di U
VT = VT[sorted_indices, :]     # Ordina le righe di VT

In [13]:
i=1

In [14]:
R_pred = np.dot(U[i, :] * Sigma, VT)  # Corrected formula

In [15]:
R_pred.shape

(69678,)

In [16]:
# Sort R_pred from highest to lowest
sorted_indices_R_pred = np.argsort(R_pred)[::-1]  # Sort in descending order
sorted_R_pred = R_pred[sorted_indices_R_pred]

# Print the top 200 sorted indices and values
for idx, (index, value) in enumerate(zip(sorted_indices_R_pred[:200], sorted_R_pred[:200])):
    print(f"Indice: {index}, Valore: {value}")

Indice: 74, Valore: 0.2879297375514102
Indice: 69, Valore: 0.2614545603505254
Indice: 70, Valore: 0.2573136856772041
Indice: 2254, Valore: 0.24091387647858192
Indice: 3824, Valore: 0.23888330010598122
Indice: 2255, Valore: 0.23698983315222164
Indice: 68, Valore: 0.22184730715171333
Indice: 2253, Valore: 0.20996403547858994
Indice: 3026, Valore: 0.207754886260024
Indice: 59, Valore: 0.20675731823342838
Indice: 71, Valore: 0.2060583016758493
Indice: 75, Valore: 0.2044666929505907
Indice: 3025, Valore: 0.1926692089035113
Indice: 4938, Valore: 0.188072090423329
Indice: 64, Valore: 0.18728253835562345
Indice: 5593, Valore: 0.1869897960511328
Indice: 4036, Valore: 0.18599395847630648
Indice: 4013, Valore: 0.18571625890110727
Indice: 3024, Valore: 0.18314539961213758
Indice: 7559, Valore: 0.17823098689223785
Indice: 5373, Valore: 0.17797906665407504
Indice: 9299, Valore: 0.17700802012177097
Indice: 11111, Valore: 0.1746767688969645
Indice: 5499, Valore: 0.16177344591026788
Indice: 5664, Valor

In [18]:
print(user_item_matrix[1].indices)
print(test_df['track'][1])

[52 55 57 58 59 60 61 62 63 64 65 67 68 69 70 71 72 74 75 76 77 78 79 80]
[73, 62]


In [55]:
print(f"Average Precision@{k}: {average_precision_at_k}")

Average Precision@10: 0.0


In [13]:
num_features = 50  # Number of latent features

# Initialize the NMF model
nmf_model = NMF(n_components=num_features, init='random', random_state=0, max_iter=100)

# Fit the model and transform the data
P = nmf_model.fit_transform(sparse_matrix)  # User features matrix
Q = nmf_model.components_       # Item features matrix

c:\Users\frmar\OneDrive\Desktop\GitHub\Music-Recommender-System\MRC\Lib\site-packages\sklearn\decomposition\_nmf.py:1741: ConvergenceWarning: Maximum number of iterations 100 reached. Increase it to improve convergence.
  warnings.warn(


In [14]:
P

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 4.50003140e-04],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [2.24395005e-05, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],
      shape=(1000001, 50))

In [15]:
np.save("P.npy", P)

print("Array saved to 'P.npy'")

Array saved to 'P.npy'


In [7]:
# Load the array from the file
P = np.load("P.npy")

print("Loaded array:")
print(P)

Loaded array:
[[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 4.50003140e-04]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [2.24395005e-05 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


In [19]:
Q

array([[0.00000000e+00, 3.26559505e-04, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.63149225e-02, 2.57897526e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.17548781e-05, 1.17548781e-05, 1.17548781e-05],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],
      shape=(50, 2262293))

In [20]:
np.save("Q.npy", Q)

print("Array saved to 'Q.npy'")

Array saved to 'Q.npy'


In [8]:
# Load the array from the file
Q = np.load("Q.npy")

print("Loaded array:")
print(Q)

Loaded array:
[[0.00000000e+00 3.26559505e-04 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.63149225e-02 2.57897526e-02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.17548781e-05
  1.17548781e-05 1.17548781e-05]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


In [11]:
import numpy as np

def predict_for_user(user_id, P, Q, interactions, k):
    """
    Predict the ratings for a specific user, excluding items they have already chosen.
    
    Parameters:
    - user_id: ID of the user (row index in P).
    - P: User feature matrix (n_users x n_components).
    - Q: Item feature matrix (n_components x n_items).
    - interactions: User-item interaction matrix (n_users x n_items) in CSR format,
                    where non-zero entries indicate the user has interacted with the item.
    
    Returns:
    - Predicted ratings for all items the user hasn't interacted with (as a dictionary).
    """
    # Validate user_id
    if user_id < 0 or user_id >= P.shape[0]:
        raise ValueError(f"Invalid user_id: {user_id}. Must be between 0 and {P.shape[0]-1}.")
    
    # Extract user features and predict ratings
    user_features = P[user_id-k]  # Features for the specific user
    predicted_ratings = user_features @ Q  # Dot product for predictions
    
    # Get the indices of items the user has interacted with (non-zero entries in CSR row)
    interacted_items = interactions[user_id].indices  # Fast access to indices of non-zero entries
    
    # Get all item indices
    all_items = np.arange(Q.shape[1])
    
    # Get indices of items the user has NOT interacted with
    unchosen_items = np.setdiff1d(all_items, interacted_items, assume_unique=True)
    
    # Filter predictions to include only unchosen items
    filtered_predictions = [predicted_ratings[item] for item in unchosen_items]
    
    return filtered_predictions

# Predict for user 1 (who has interacted with items 1 and 2)
user_id = 999999
predicted_ratings_user_1 = predict_for_user(user_id, P, Q, sparse_matrix, 5)


In [12]:
max_index = predicted_ratings_user_1.index(max(predicted_ratings_user_1))
print(max_index)

3546


In [13]:
len(predicted_ratings_user_1)

2262189

In [7]:
# Step 3: Apply Truncated SVD
import numpy as np
from scipy.sparse import coo_matrix
from sklearn.decomposition import TruncatedSVD

# Convert COO to CSR for better compatibility
csr = sparse_matrix.tocsr()
csr = csr.astype(float)

from scipy.sparse.linalg import svds

# Perform Truncated SVD with a limited number of components
k = 50  # Number of components
U, sigma, Vt = svds(csr, k=k)

print("U Matrix:", U)
print("Singular Values:", sigma)
print("Vt Matrix:", Vt)

U Matrix: [[ 7.91152764e-05  4.55646943e-04 -5.02736905e-04 ... -1.44587858e-03
   1.82402848e-03  7.74491536e-04]
 [ 1.75035423e-05  3.12490426e-05  8.15832347e-05 ...  1.92904045e-03
   8.90942199e-04  1.31713749e-04]
 [ 5.69781061e-05  4.32067831e-05  1.12561262e-05 ... -1.92183629e-06
   2.77222113e-06  4.10188747e-06]
 ...
 [ 1.49053367e-05  1.33442312e-06  1.13877515e-05 ... -1.33526936e-06
   6.66563146e-06  2.76084294e-06]
 [ 9.32822601e-05 -1.24941083e-05 -4.67362381e-05 ... -4.75555934e-06
   3.69669216e-06  5.87576196e-06]
 [ 6.13680740e-20  3.31500616e-20 -1.78220890e-19 ... -1.23308129e-19
   1.10936867e-19  1.57812097e-19]]
Singular Values: [209.28778874 210.28093996 212.42397963 213.19873828 218.01344851
 219.65115984 221.54174498 223.27100752 224.37757141 226.87397191
 231.72266705 232.93402124 234.09808801 235.74833292 242.61872012
 243.18283765 249.04423834 249.49920643 252.33963985 260.88655205
 262.95955062 266.90232618 269.17892512 274.47680879 274.89211621
 281.16

In [8]:
print(f'Dimensione U: {U.shape}')
print(f'Dimensione sigma: {sigma.shape}')
print(f'Dimensione Vt: {Vt.shape}')

Dimensione U: (1000001, 50)
Dimensione sigma: (50,)
Dimensione Vt: (50, 2262293)


In [9]:
# Convert sigma to diagonal matrix form
sigma = np.diag(sigma)

In [10]:
sigma

array([[209.28778874,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        , 210.28093996,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        , 212.42397963, ...,   0.        ,
          0.        ,   0.        ],
       ...,
       [  0.        ,   0.        ,   0.        , ..., 596.02062965,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
        690.3306173 ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        , 906.25404211]], shape=(50, 50))

In [37]:
user = 999999

R_pred_user = np.dot(np.dot(U[user, :k], sigma[:k, :k]), Vt[:k, :])

print(R_pred_user)

[ 6.85434061e-17 -5.95851875e-04 -1.00930750e-03 ... -2.43592414e-08
 -2.43592414e-08 -2.43592414e-08]


In [38]:
user_tracks = data.loc[data['user_id'] == user, 'tracks'].values[0]
user_tracks = user_tracks[:-1]
# Print the tracks for user i
print(f"Tracks for user {user}: {user_tracks}")

Tracks for user 999999: [12421, 12518, 14294, 268821, 311299, 92033, 12463, 41601, 12423, 41168, 39990, 12422, 41184, 273644, 97741, 72435, 314025, 12556, 189369, 72395, 153573, 39988, 40000, 14304, 12426, 12425, 153550, 462400, 12493, 12424, 45776, 12451, 12455, 138207, 379395, 83427, 72425, 40017, 13644, 12489, 12552, 274131, 168841, 194398, 125607, 12444, 71112, 189377, 263925, 377469, 133842, 12461, 14260, 163478, 12535, 153545, 363031, 69331, 39982, 77299, 12511, 12512, 12462, 153537, 153538, 97742, 53263, 153539, 153540, 153541, 153542, 153543, 153544, 12462, 12420, 40001, 84864, 311291, 45751, 270010, 125617, 582309, 582312, 12424, 627056, 3897, 83428, 340556, 194410, 41163, 12489, 263925, 101245, 327267, 250248, 250249, 128315, 221578, 12426, 41167, 153549, 466735, 466733, 72395, 72411, 72433, 288295, 141055, 141040]


In [39]:
new_list = [item for idx, item in enumerate(R_pred_user) if idx not in user_tracks]

In [40]:
max_position = np.argmax(new_list)
print(max_position)

5313
